# 線形最適化

In [1]:
from pulp import *
prob = LpProblem(name = 'LP-sample', sense=LpMaximize)

# 変数の設定
x1 = LpVariable('x1', lowBound=0.0)
x2 = LpVariable('x2', lowBound=0.0)

# 目的関数の設定
prob += 2*x1 + 3*x2

# 条件の設定
prob += x1 + 3*x2 <= 9, 'ineq1'
prob += x1 + x2 <= 4, 'ineq2'
prob += x1 + x2 <= 6, 'ineq3'

# 問題を出力
print('probを表示します...')
print(prob)

# 計算
prob.solve()

#結果を表示
print('Statusを表示します...')
print(LpStatus[prob.status])
print('')

print('結果を表示します...')
print('Optimal value=', value(prob.objective))
for v in prob.variables():
    print(v.name, '=', value(v))

probを表示します...
LP-sample:
MAXIMIZE
2*x1 + 3*x2 + 0
SUBJECT TO
ineq1: x1 + 3 x2 <= 9

ineq2: x1 + x2 <= 4

ineq3: x1 + x2 <= 6

VARIABLES
x1 Continuous
x2 Continuous

Statusを表示します...
Optimal

結果を表示します...
Optimal value= 10.5
x1 = 1.5
x2 = 2.5


# 線形最適化(numpyを使って式を定義)

In [2]:
#係数をnumpyで指定する

from pulp import *
import numpy as np

#numpyで係数を指定する
#目的関数：cx
#条件：Ax<=b

A = np.array([[3, 1, 2],
             [1, 3, 0],
             [0, 2, 4]])
c = np.array([150, 200, 300])
b = np.array([60, 36, 48])

(m, n) = A.shape
prob = LpProblem(name='Production', sense=LpMaximize)
x = [LpVariable('x'+str(i+1), lowBound = 0) for i in range(n)]
prob += lpDot(c,x)

for i in range(m):
    prob += lpDot(A[i], x) <= b[i], 'ineq'+str(i)
print(prob)
prob.solve()

print(LpStatus[prob.status])
print('Optimal value=', value(prob.objective))
for v in prob.variables():
    print(v.name, '=', value(v))

Production:
MAXIMIZE
150*x1 + 200*x2 + 300*x3 + 0
SUBJECT TO
ineq0: 3 x1 + x2 + 2 x3 <= 60

ineq1: x1 + 3 x2 <= 36

ineq2: 2 x2 + 4 x3 <= 48

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

Optimal
Optimal value= 5800.0
x1 = 12.0
x2 = 8.0
x3 = 8.0


In [3]:
# 不等式制約を満たすことを確認する
X = np.array([v.varValue for v in prob.variables()])
X

array([12.,  8.,  8.])

In [4]:
# すべてTrueであればTrueになる
np.all(np.abs(b - np.dot(A,X)) <= 1.0e-5)

True

# 双対問題を解く

In [7]:
# Aを転置
AT = A.T

# 最小化問題
dual = LpProblem(name='Dual_Production', sense=LpMinimize)

# 変数を設定
y = [LpVariable('y'+str(i+1), lowBound=0) for i in range(m)]

# 目的関数を設定
dual += lpDot(b,y)

# 条件を設定
for j in range(n):
    dual += lpDot(AT[j],y) >= c[j], 'ineq'+str(j)

# 計算
dual.solve()
print(LpStatus[dual.status])

# 結果を表示
print('Optimal value of dual problem =', value(dual.objective))
for v in dual.variables():
    print(v.name, '=', v.varValue)

Optimal
Optimal value of dual problem = 5799.999996
y1 = 44.444444
y2 = 16.666667
y3 = 52.777778


In [8]:
# 不等式制約を満たすことを確認する
Y=np.array([v.varValue for v in dual.variables()])
np.all(np.abs(np.dot(AT,Y) - c) <= 1.0e-5)

True